## 라이브러리

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import itertools

## 데이터 불러오기 및 전처리

In [3]:
df = pd.read_csv('/content/drive/MyDrive/DATATHON/data/Lotte/lotte_hitter_2024.csv', encoding='cp949')
columns_to_keep = ['선수명', '팀명', '타율', '경기', '타수', '홈런', '득점', '장타율', '출루율', '득점권타율', '고의4구', '희생플라이', '희생번트']
df = df[columns_to_keep]

df3 = pd.read_csv('/content/drive/MyDrive/DATATHON/data/Lotte/lotte_runner_2024.csv', encoding='cp949')

df3 = df3[['선수명', '도루허용', '도루저지']]

df = pd.merge(df, df3, on='선수명', how='left')

# URL 지정 (스탯티즈에서 수비 승리 기여도 가져오기)
url = "https://statiz.sporki.com/stats/?m=main&m2=fielding&m3=default&so=WAAwithPOS&ob=DESC&year=2024&sy=&ey=&te=3001&po=&lt=10100&reg=A&pe=&ds=&de=&we=&hr=&ha=&ct=&st=&vp=&bo=&pt=&pp=&ii=&vc=&um=&oo=&rr=&sc=&bc=&ba=&li=&as=&ae=&pl=&gc=&lr=&pr=100&ph=&hs=&us=&na=&ls=0&sf1=G&sk1=&sv1=&sf2=G&sk2=&sv2="

# 웹페이지에서 테이블 읽기
dfs = pd.read_html(url)

# 첫 번째 테이블을 데이터프레임으로 선택
df5 = dfs[0]

df5.to_csv('New_defense.csv', index=False)

df5 = pd.read_csv('New_defense.csv')

#### 컬럼정리 ####
df5= df5[df5['Rank'] != 'Rank']

# 컬럼명 한글이름으로 바꾸기
columns_to_keep = ['Name', 'Team', 'Sort▼', 'IP']
df5 = df5[columns_to_keep].rename(columns={'Sort▼': '수비 승리 기여도', 'Name': '선수명', 'Team': '포지션',
                                            'IP' : '수비이닝'})

# 포지션 컬럼에서 앞 세 글자 제외
df5['포지션'] = df5['포지션'].str[3:]

position_mapping = {
    '1B': '1루수',    '2B': '2루수',    '3B': '3루수',    'SS': '유격수',
    'C': '포수',    'RF': '우익수',    'CF': '중견수',    'LF': '좌익수'
}

# 포지션 컬럼 값 변환
df5['포지션'] = df5['포지션'].map(position_mapping)

# '경기'와 '수비이닝' 컬럼을 float로 변환 (잘못된 형식이 있을 경우를 대비하여 정리)
df5['수비이닝'] = pd.to_numeric(df5['수비이닝'], errors='coerce')
df5['수비 승리 기여도'] = pd.to_numeric(df5['수비 승리 기여도'], errors='coerce')

# '포지션' 열의 NaN 값을 빈 문자열로 대체하고 문자열로 변환
df5['포지션'] = df5['포지션'].fillna('').astype(str)

# 수비이닝을 기준으로 포지션 정렬 함수
def aggregate_positions(group):
    sorted_positions = group.sort_values(by='수비이닝', ascending=False)
    return ', '.join(sorted_positions['포지션'])

# 선수명을 기준으로 그룹화하여 수비 승리 기여도의 평균을 계산하고 포지션을 결합
df5 = df5.groupby('선수명').apply(
    lambda x: pd.Series({
        '포지션': aggregate_positions(x),
        '수비 승리 기여도': x['수비 승리 기여도'].mean(),
        '수비이닝': x['수비이닝'].sum()
    })
).reset_index()

df = pd.merge(df, df5, on='선수명', how='left')
df = df.dropna()

df = df[df['경기'] >= 10]
df = df[df['타수'] >= 31]

df

,선수명,팀명,타율,경기,타수,홈런,득점,장타율,출루율,득점권타율,고의4구,희생플라이,희생번트,도루허용,도루저지,포지션,수비 승리 기여도,수비이닝
2,황성빈,롯데,0.354,59,161,4,48,0.491,0.409,0.296,0,0,1,30,3,"중견수, 좌익수, 우익수",0.014667,339.2
3,레이예스,롯데,0.337,74,294,7,39,0.486,0.371,0.398,0,7,0,4,0,"우익수, 좌익수, 중견수",-0.152333,491.4
4,손호영,롯데,0.324,48,176,8,31,0.551,0.362,0.382,1,1,1,6,4,"3루수, 2루수, 유격수",-0.172333,336.3
6,전준우,롯데,0.314,40,159,7,20,0.528,0.375,0.255,0,1,0,3,1,좌익수,-0.269000,149.1
7,고승민,롯데,0.309,54,204,5,37,0.466,0.378,0.339,0,4,0,4,2,"2루수, 좌익수, 우익수, 1루수",0.025750,412.1
8,윤동희,롯데,0.305,71,266,5,59,0.451,0.375,0.348,0,3,1,5,2,"중견수, 우익수",-0.130000,571.0
9,나승엽,롯데,0.298,51,171,1,23,0.456,0.381,0.317,0,2,0,0,0,1루수,-0.601000,371.2
10,박승욱,롯데,0.273,72,187,4,27,0.396,0.344,0.217,0,1,2,1,4,"유격수, 3루수, 2루수",-0.183000,464.2
11,이학주,롯데,0.269,37,93,2,13,0.398,0.320,0.158,0,0,2,0,1,"유격수, 3루수",-0.206000,233.3
12,한동희,롯데,0.257,14,35,0,1,0.314,0.278,0.182,1,0,0,0,0,3루수,-0.297000,32.1


batting order에 도루허용, 도루저지, 고의4구, 희생플라이, 희생번트 값 넣어서 추정득점 구하기

In [4]:
df2 = pd.read_csv('/content/drive/MyDrive/DATATHON/data/Lotte/lotte_batting_order_2024.csv', encoding='cp949')

filtered_names = df['선수명'].unique()
df2 = df2[df2['선수명'].isin(filtered_names)]

# 1루타 열 계산 (안타 - 2루타 - 3루타 - 홈런)
df2['1루타'] = df2['안타'] - df2['2루타'] - df2['3루타'] - df2['홈런']
df2 = pd.merge(df2, df[['선수명', '도루허용', '도루저지', '고의4구', '희생플라이', '희생번트', '수비 승리 기여도', '득점권타율']], on='선수명', how='left')

# XR 계산 함수 정의
def calculate_XR(row):
    XR = (row['1루타'] * 0.5 +
          row['2루타'] * 0.72 +
          row['3루타'] * 1.04 +
          row['홈런'] * 1.44 +
          (row['사구'] + row['볼넷'] - row['고의4구']) * 0.34 +
          row['고의4구'] * 0.25 +
          row['도루허용'] * 0.18 -
          row['도루저지'] * 0.32 -
          (row['타수'] - row['안타'] - row['삼진']) * 0.09 -
          row['삼진'] * 0.098 -
          row['병살타'] * 0.37 +
          row['희생플라이'] * 0.37 +
          row['희생번트'] * 0.04)  # 희생번트 값 추가
    return XR

# 출루율 계산 함수 정의
def calculate_obp(row):
    hits = row['안타']
    walks = row['볼넷']
    hbp = row['사구']
    at_bats = row['타수']
    sac_flies = row['희생플라이']
    obp = (hits + walks + hbp) / (at_bats + walks + hbp + sac_flies) if (at_bats + walks + hbp + sac_flies) > 0 else 0
    return obp

# 장타율 계산 함수 정의
def calculate_slg(row):
    singles = row['안타'] - row['2루타'] - row['3루타'] - row['홈런']
    doubles = row['2루타']
    triples = row['3루타']
    homers = row['홈런']
    at_bats = row['타수']
    slg = (singles + 2*doubles + 3*triples + 4*homers) / at_bats if at_bats > 0 else 0
    return slg

# 각 선수의 XR 계산
df2['추정득점'] = df2.apply(calculate_XR, axis=1)
df2['출루율'] = df2.apply(calculate_obp, axis=1)
df2['장타율'] = df2.apply(calculate_slg, axis=1)

# '순위' 컬럼 제외
df2.drop(columns=['순위'], axis=1, inplace=True)

# '상위(1~2번)', '중심(3~5번)', '하위(6~9번)' 타순 제외
excluded_orders = ['상위(1~2번)', '중심(3~5번)', '하위(6~9번)']
df2 = df2[~df2['타순'].isin(excluded_orders)]

# '타율' column을 float 타입으로 변환
df2['타율'] = df2['타율'].replace('-', np.nan).astype(float)

# '타수'가 10개 이상인 선수들만 선택
df2 = df2[df2['타수']>= 10]

# 결과 출력
df2

,선수명,팀명,타율,타수,안타,2루타,3루타,홈런,타점,볼넷,...,도루허용,도루저지,고의4구,희생플라이,희생번트,수비 승리 기여도,득점권타율,추정득점,출루율,장타율
1,황성빈,롯데,0.318,107,34,0,2,1,3,11,...,30,3,0,0,1,0.014667,0.296,19.826,0.381356,0.383178
2,윤동희,롯데,0.264,159,42,10,1,3,17,22,...,5,2,0,3,1,-0.130000,0.348,23.956,0.347826,0.396226
3,김민석,롯데,0.067,15,1,0,0,0,0,1,...,2,1,0,0,1,-0.034000,0.300,0.656,0.263158,0.066667
9,황성빈,롯데,0.542,24,13,0,2,3,8,3,...,30,3,0,0,1,0.014667,0.296,14.886,0.592593,1.083333
11,윤동희,롯데,0.394,71,28,5,2,0,13,5,...,5,2,0,3,1,-0.130000,0.348,15.302,0.425000,0.521127
12,고승민,롯데,0.292,120,35,9,1,4,15,13,...,4,2,0,4,0,0.025750,0.339,21.904,0.355072,0.483333
16,김민석,롯데,0.235,17,4,0,1,0,2,0,...,2,1,0,0,1,-0.034000,0.300,0.678,0.235294,0.352941
17,정훈,롯데,0.233,43,10,3,0,2,4,4,...,1,0,0,1,0,0.032333,0.240,6.046,0.291667,0.441860
21,레이예스,롯데,0.335,158,53,9,1,5,29,14,...,4,0,0,7,0,-0.152333,0.398,30.638,0.377778,0.500000
22,손호영,롯데,0.326,43,14,1,0,3,12,2,...,6,4,1,1,1,-0.172333,0.382,8.514,0.361702,0.558140


In [5]:
# 가중 평균 계산 함수
def calculate_weighted_average(records):
    weighted_avg = {}
    total_weight = records['타수'].sum()
    for column in records.columns:
        if column not in ['선수명', '팀명', '타순'] and pd.api.types.is_numeric_dtype(records[column]):
            weighted_avg[column] = (records[column] * records['타수']).sum() / total_weight
    return weighted_avg

# 각 선수에 대해 처리
unique_players = df2['선수명'].unique()
all_new_records = []

for player in unique_players:
    player_records = df2[df2['선수명'] == player]
    existing_orders = player_records['타순'].unique()
    all_orders = [f'{i}번' for i in range(1, 10)]
    missing_orders = [order for order in all_orders if order not in existing_orders]

    for order in missing_orders:
        weighted_avg = calculate_weighted_average(player_records)
        weighted_avg['타율'] = weighted_avg['안타'] / weighted_avg['타수'] if weighted_avg['타수'] != 0 else 0
        weighted_avg['선수명'] = player
        weighted_avg['팀명'] = player_records['팀명'].iloc[0]
        weighted_avg['타순'] = order
        all_new_records.append(weighted_avg)

# 새로운 기록들을 DataFrame으로 변환하고 기존 df2에 추가
new_records_df = pd.DataFrame(all_new_records)
df3 = pd.concat([df2, new_records_df], ignore_index=True)

# 기존 타율이 계산된 후에도 유지되도록 설정
df3['타율'] = df3.apply(lambda row: row['안타'] / row['타수'] if row['타수'] != 0 else row['타율'], axis=1)

# 타수 조정
df3 = df3[df3['타수'] >= 10]

# 소수점 셋째 자리까지 반올림할 열 목록
decimal_columns = ['타율', '수비 승리 기여도', '득점권타율', '추정득점', '출루율', '장타율']

# 소수점 없이 정수로 표시할 열 목록
integer_columns = ['타수', '안타', '2루타', '3루타', '홈런', '타점', '볼넷', '사구', '삼진', '병살타', '1루타', '도루허용', '도루저지', '고의4구', '희생플라이', '희생번트']

# 숫자 데이터를 소수점 셋째 자리까지 반올림
df3[decimal_columns] = df3[decimal_columns].round(3)

# NaN 값을 0으로 대체
df3[integer_columns] = df3[integer_columns].fillna(0)

# 숫자 데이터를 정수로 변환
df3[integer_columns] = df3[integer_columns].astype(int)

# 결과 출력
df2 = df3
df2

,선수명,팀명,타율,타수,안타,2루타,3루타,홈런,타점,볼넷,...,도루허용,도루저지,고의4구,희생플라이,희생번트,수비 승리 기여도,득점권타율,추정득점,출루율,장타율
0,황성빈,롯데,0.318,107,34,0,2,1,3,11,...,30,3,0,0,1,0.015,0.296,19.826,0.381,0.383
1,윤동희,롯데,0.264,159,42,10,1,3,17,22,...,5,2,0,3,1,-0.130,0.348,23.956,0.348,0.396
2,김민석,롯데,0.067,15,1,0,0,0,0,1,...,2,1,0,0,1,-0.034,0.300,0.656,0.263,0.067
3,황성빈,롯데,0.542,24,13,0,2,3,8,3,...,30,3,0,0,1,0.015,0.296,14.886,0.593,1.083
4,윤동희,롯데,0.394,71,28,5,2,0,13,5,...,5,2,0,3,1,-0.130,0.348,15.302,0.425,0.521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,정보근,롯데,0.210,37,7,0,0,1,1,0,...,0,0,0,0,2,0.090,0.176,2.067,0.230,0.312
158,정보근,롯데,0.210,37,7,0,0,1,1,0,...,0,0,0,0,2,0.090,0.176,2.067,0.230,0.312
159,정보근,롯데,0.210,37,7,0,0,1,1,0,...,0,0,0,0,2,0.090,0.176,2.067,0.230,0.312
160,정보근,롯데,0.210,37,7,0,0,1,1,0,...,0,0,0,0,2,0.090,0.176,2.067,0.230,0.312


## 라인업 만들기

In [6]:
df.replace('-', np.nan, inplace=True)

df['장타율'] = df['장타율'].astype(float)
df['출루율'] = df['출루율'].astype(float)
df['타율'] = df['타율'].astype(float)

### 중심타선

In [7]:
# 표준화 스케일러 생성
scaler = StandardScaler()

# 장타율과 득점권 타율 표준화
df[['장타율_표준화', '득점권 타율_표준화', '홈런_표준화']] = scaler.fit_transform(df[['장타율', '득점권타율', '홈런']])

# 표준화된 지표 합산
df['중심타선 합산 지표'] = df['장타율_표준화']*0.46 + df['득점권 타율_표준화']*0.33 + df['홈런_표준화']*0.56

# 표준화된 합산 지표 기준으로 상위 3명 선택
center_hitters_1 = df

# 결과 출력
center_hitters_1 = center_hitters_1.nlargest(3, '중심타선 합산 지표')
center_hitters_1.nlargest(3, '중심타선 합산 지표')

,선수명,팀명,타율,경기,타수,홈런,득점,장타율,출루율,득점권타율,...,희생번트,도루허용,도루저지,포지션,수비 승리 기여도,수비이닝,장타율_표준화,득점권 타율_표준화,홈런_표준화,중심타선 합산 지표
4,손호영,롯데,0.324,48,176,8,31,0.551,0.362,0.382,...,1,6,4,"3루수, 2루수, 유격수",-0.172333,336.3,1.611991,1.537173,1.838290,2.278225
3,레이예스,롯데,0.337,74,294,7,39,0.486,0.371,0.398,...,0,4,0,"우익수, 좌익수, 중견수",-0.152333,491.4,0.949296,1.719057,1.444371,1.812813
6,전준우,롯데,0.314,40,159,7,20,0.528,0.375,0.255,...,0,3,1,좌익수,-0.269000,149.1,1.377499,0.093468,1.444371,1.473342


In [8]:
# center_hitters의 선수명을 df2에서 찾기
center_hitters_names = center_hitters_1['선수명'].values
filtered_df2 = df2[df2['선수명'].isin(center_hitters_names)]

# '타순'이 '3번', '4번', '5번'인 것만 남기기
filtered_df2 = filtered_df2[filtered_df2['타순'].isin(['3번', '4번', '5번'])]

center_hitters = filtered_df2

# center_hitters_1에서 '포지션' 컬럼만 선택
center_hitters_1_position = center_hitters_1[['선수명', '포지션']]

# '선수명'을 기준으로 병합
center_hitters = pd.merge(center_hitters, center_hitters_1_position, on='선수명', how='left')

center_hitters

,선수명,팀명,타율,타수,안타,2루타,3루타,홈런,타점,볼넷,...,도루저지,고의4구,희생플라이,희생번트,수비 승리 기여도,득점권타율,추정득점,출루율,장타율,포지션
0,레이예스,롯데,0.335,158,53,9,1,5,29,14,...,0,0,7,0,-0.152,0.398,30.638,0.378,0.500,"우익수, 좌익수, 중견수"
1,손호영,롯데,0.326,43,14,1,0,3,12,2,...,4,1,1,1,-0.172,0.382,8.514,0.362,0.558,"3루수, 2루수, 유격수"
2,레이예스,롯데,0.338,136,46,10,1,2,32,5,...,0,0,7,0,-0.152,0.398,23.268,0.345,0.471,"우익수, 좌익수, 중견수"
3,전준우,롯데,0.318,154,49,11,1,6,30,14,...,1,0,1,0,-0.269,0.255,27.992,0.380,0.519,좌익수
4,손호영,롯데,0.333,60,20,5,1,3,13,4,...,4,1,1,1,-0.172,0.382,11.860,0.379,0.600,"3루수, 2루수, 유격수"
5,레이예스,롯데,0.337,147,49,9,1,3,30,9,...,0,0,7,0,-0.152,0.398,27.229,0.362,0.486,"우익수, 좌익수, 중견수"
6,손호영,롯데,0.345,53,18,3,0,2,12,2,...,4,1,1,1,-0.172,0.382,10.598,0.373,0.592,"3루수, 2루수, 유격수"
7,전준우,롯데,0.318,154,49,11,1,6,30,14,...,1,0,1,0,-0.269,0.255,27.992,0.380,0.519,좌익수
8,전준우,롯데,0.318,154,49,11,1,6,30,14,...,1,0,1,0,-0.269,0.255,27.992,0.380,0.519,좌익수


In [9]:
# 각 선수명에 따른 타순 값을 리스트로 모음
center_per_player = filtered_df2.groupby('선수명')['타순'].apply(list).reset_index()
center_per_player

,선수명,타순
0,레이예스,"[3번, 4번, 5번]"
1,손호영,"[3번, 5번, 4번]"
2,전준우,"[4번, 3번, 5번]"


### 2번타자

In [10]:
# 표준화 스케일러 생성
scaler = StandardScaler()

# 장타율, 출루율, 도루허용 표준화
df[['장타율_표준화', '출루율_표준화', '도루_표준화', '홈런_표준화']] = scaler.fit_transform(df[['장타율', '출루율', '도루허용', '홈런']])

# 2번타자 지표 합산 (가중치는 예시로 설정, 필요에 따라 조정 가능)
df['2번타자 합산 지표'] = df['장타율_표준화']*0.46 + df['출루율_표준화']*0.35 + df['도루_표준화']*0.33 + df['홈런_표준화']*0.56

# center_hitters에 있는 선수명 제외 (중심타선 3명 제외)
exclude_names = center_hitters['선수명'].tolist()
filtered_df = df[~df['선수명'].isin(exclude_names)]

# center_hitters의 주 포지션 확인
center_main_positions = center_hitters['포지션'].apply(lambda x: x.split(',')[0]).unique()

# 포지션별 필터링 함수
def filter_by_position(df, position, min_count, max_count, excluded_positions):
    if position in excluded_positions:
        return pd.DataFrame()  # 제외된 포지션이면 빈 데이터프레임 반환
    pos_df = df[df['포지션'].str.contains(position)]
    if len(pos_df) > max_count:
        pos_df = pos_df.nlargest(max_count, '2번타자 합산 지표')
    elif len(pos_df) < min_count:
        pos_df = df[df['포지션'].str.contains(position)].nlargest(min_count, '2번타자 합산 지표')
    return pos_df

# 포지션별로 필터링
catcher = filter_by_position(filtered_df, '포수', 1, 1, center_main_positions)
outfielders = filter_by_position(filtered_df, '좌익수|우익수|중견수', 1, 3, center_main_positions)
infielders = filter_by_position(filtered_df, '1루수|2루수|3루수|유격수', 1, 4, center_main_positions)

# 최종 데이터프레임 합치기
final_candidates = pd.concat([catcher, outfielders, infielders]).drop_duplicates()

# # 2번타자 합산 지표 기준으로 상위 1명 선택
second_hitter_1 = final_candidates.nlargest(1, '2번타자 합산 지표')
second_hitter_1

,선수명,팀명,타율,경기,타수,홈런,득점,장타율,출루율,득점권타율,...,포지션,수비 승리 기여도,수비이닝,장타율_표준화,득점권 타율_표준화,홈런_표준화,중심타선 합산 지표,출루율_표준화,도루_표준화,2번타자 합산 지표
2,황성빈,롯데,0.354,59,161,4,48,0.491,0.409,0.296,...,"중견수, 좌익수, 우익수",0.014667,339.2,1.000273,0.559546,0.262613,0.791839,1.581618,3.95079,2.464516


In [11]:
# second_hitter의 선수명을 df2에서 찾기
second_hitter_names = second_hitter_1['선수명'].values
filtered_df2 = df2[df2['선수명'].isin(second_hitter_names)]

# '타순'이 '2번' 인 것만 남기기
filtered_df2 = filtered_df2[filtered_df2['타순'].isin(['2번'])]

second_hitter = filtered_df2
# second_hitters_1에서 '포지션' 컬럼만 선택
second_hitter_1_position = second_hitter_1[['선수명', '포지션']]

# '선수명'을 기준으로 병합
second_hitter = pd.merge(second_hitter, second_hitter_1_position, on='선수명', how='left')
second_hitter

,선수명,팀명,타율,타수,안타,2루타,3루타,홈런,타점,볼넷,...,도루저지,고의4구,희생플라이,희생번트,수비 승리 기여도,득점권타율,추정득점,출루율,장타율,포지션
0,황성빈,롯데,0.542,24,13,0,2,3,8,3,...,3,0,0,1,0.015,0.296,14.886,0.593,1.083,"중견수, 좌익수, 우익수"


### 1번 타자

In [12]:
# 표준화 스케일러 생성
scaler = StandardScaler()

# 장타율, 출루율, 도루허용 표준화
df[['출루율_표준화', '도루_표준화']] = scaler.fit_transform(df[['출루율', '도루허용']])

# 1번타자 지표 합산 (가중치는 예시로 설정, 필요에 따라 조정 가능)
df['1번타자 합산 지표'] = df['출루율_표준화']*0.35 + df['도루_표준화']*0.33

# center_hitters와 second_hitter_1에 있는 선수명 제외
exclude_names = center_hitters['선수명'].tolist() + second_hitter_1['선수명'].tolist()
filtered_df = df[~df['선수명'].isin(exclude_names)]

# center_hitters와 second_hitter_1의 주 포지션 확인
center_main_positions = center_hitters['포지션'].apply(lambda x: x.split(',')[0]).unique()
second_hitter_main_positions = second_hitter_1['포지션'].apply(lambda x: x.split(',')[0]).unique()
excluded_positions = list(center_main_positions) + list(second_hitter_main_positions)

# 포지션별 필터링 함수
def filter_by_position(df, position, min_count, max_count, excluded_positions):
    if position in excluded_positions:
        return pd.DataFrame()  # 제외된 포지션이면 빈 데이터프레임 반환
    pos_df = df[df['포지션'].str.contains(position)]
    if len(pos_df) > max_count:
        pos_df = pos_df.nlargest(max_count, '1번타자 합산 지표')
    elif len(pos_df) < min_count:
        pos_df = df[df['포지션'].str.contains(position)].nlargest(min_count, '1번타자 합산 지표')
    return pos_df

# 포지션별로 필터링
catcher = filter_by_position(filtered_df, '포수', 1, 1, excluded_positions)
outfielders = filter_by_position(filtered_df, '좌익수|우익수|중견수', 1, 3, excluded_positions)
infielders = filter_by_position(filtered_df, '1루수|2루수|3루수|유격수', 1, 4, excluded_positions)

# 최종 데이터프레임 합치기
final_candidates = pd.concat([catcher, outfielders, infielders]).drop_duplicates()

# 1번타자 합산 지표 기준으로 상위 1명 선택
first_hitter_1 = final_candidates.nlargest(1, '1번타자 합산 지표')
first_hitter_1

,선수명,팀명,타율,경기,타수,홈런,득점,장타율,출루율,득점권타율,...,수비 승리 기여도,수비이닝,장타율_표준화,득점권 타율_표준화,홈런_표준화,중심타선 합산 지표,출루율_표준화,도루_표준화,2번타자 합산 지표,1번타자 합산 지표
8,윤동희,롯데,0.305,71,266,5,59,0.451,0.375,0.348,...,-0.13,571.0,0.59246,1.150669,0.656532,1.019911,0.981153,0.277535,1.07518,0.43499


In [13]:
# first_hitters의 선수명을 df2에서 찾기
first_hitter_names = first_hitter_1['선수명'].values
filtered_df2 = df2[df2['선수명'].isin(first_hitter_names)]

# '타순'이 '1번' 인 것만 남기기
filtered_df2 = filtered_df2[filtered_df2['타순'].isin(['1번'])]

first_hitter = filtered_df2
# first_hitter_1에서 '포지션' 컬럼만 선택
first_hitter_1_position = first_hitter_1[['선수명', '포지션']]

# '선수명'을 기준으로 병합
first_hitter = pd.merge(first_hitter, first_hitter_1_position, on='선수명', how='left')

first_hitter

,선수명,팀명,타율,타수,안타,2루타,3루타,홈런,타점,볼넷,...,도루저지,고의4구,희생플라이,희생번트,수비 승리 기여도,득점권타율,추정득점,출루율,장타율,포지션
0,윤동희,롯데,0.264,159,42,10,1,3,17,22,...,2,0,3,1,-0.13,0.348,23.956,0.348,0.396,"중견수, 우익수"


### 하위타선

In [14]:
# 출루율과 수비 승리 기여도 표준화
scaler = StandardScaler()
df[['출루율_표준화', '수비_표준화']] = scaler.fit_transform(df[['출루율', '수비 승리 기여도']])

# 하위타선 지표 합산
df['하위타선 합산 지표'] = df['출루율_표준화']*0.35 + df['수비_표준화']*0.10

# center_hitters, second_hitter_1, first_hitter_1에 있는 선수명 제외
exclude_names = center_hitters['선수명'].tolist() + second_hitter_1['선수명'].tolist() + first_hitter_1['선수명'].tolist()
filtered_df = df[~df['선수명'].isin(exclude_names)]

# center_hitters, second_hitter_1, first_hitter_1의 주 포지션 확인
center_main_positions = center_hitters['포지션'].apply(lambda x: x.split(',')[0]).unique()
second_hitter_main_positions = second_hitter_1['포지션'].apply(lambda x: x.split(',')[0]).unique()
first_hitter_main_positions = first_hitter_1['포지션'].apply(lambda x: x.split(',')[0]).unique()
excluded_positions = list(center_main_positions) + list(second_hitter_main_positions) + list(first_hitter_main_positions)

# 포지션별 필터링 함수
def filter_by_position(df, position, excluded_positions):
    if position in excluded_positions:
        return pd.DataFrame()  # 제외된 포지션이면 빈 데이터프레임 반환
    return df[df['포지션'].str.contains(position)]

# 포지션별로 필터링
catchers = filter_by_position(filtered_df, '포수', excluded_positions)
outfielders = filter_by_position(filtered_df, '좌익수|우익수|중견수', excluded_positions)
infielders = filter_by_position(filtered_df, '1루수|2루수|3루수|유격수', excluded_positions)

# 상위순으로 나열하기
bottom_candidates = pd.concat([catchers, outfielders, infielders]).drop_duplicates()

# 포수가 center_hitters, second_hitter_1, first_hitter_1에 없는지 확인
if not any('포수' in pos for pos in center_main_positions) and \
   not any('포수' in pos for pos in second_hitter_main_positions) and \
   not any('포수' in pos for pos in first_hitter_main_positions):
    if '포수' not in bottom_candidates['포지션'].values:
        # 포수를 제외한 나머지 선수들 중 상위 3명 선택
        bottom_candidates = bottom_candidates.iloc[:-1]
        # filtered_df에서 포지션이 포수인 선수 추가
        catcher = filtered_df[filtered_df['포지션'].str.contains('포수')].nlargest(1, '하위타선 합산 지표')
        bottom_candidates = pd.concat([bottom_candidates, catcher])

# 포수가 2명 이상 포함된 경우 처리
if bottom_candidates['포지션'].str.contains('포수').sum() > 1:
    # 포수들 중에서 하위타선 합산 지표가 가장 높은 선수 선택
    best_catcher = bottom_candidates[bottom_candidates['포지션'].str.contains('포수')].nlargest(1, '하위타선 합산 지표')
    # 포수를 제외한 나머지 선수들 중 상위 3명 선택
    bottom_candidates = bottom_candidates[~bottom_candidates['포지션'].str.contains('포수')].nlargest(3, '하위타선 합산 지표')
    # 다시 포수 추가하여 상위 4명 선택
    bottom_candidates = pd.concat([bottom_candidates, best_catcher])

# 결과 출력
bottom_hitters_1 = bottom_candidates.nlargest(4, '하위타선 합산 지표').drop_duplicates()

# 앞에서 선택한 선수들의 포지션
selected_positions = set(center_main_positions) | set(second_hitter_main_positions) | set(first_hitter_main_positions)

# 부족한 포지션이 있는지 확인
missing_positions = {'포수', '좌익수', '우익수', '중견수', '1루수', '2루수', '3루수', '유격수'} - selected_positions

# 부족한 포지션이 있다면 처리
if missing_positions:
    for position in missing_positions:
        if not any(position in pos for pos in bottom_hitters_1['포지션']):
            # 점수가 낮은 선수 제거
            lowest_score_player = bottom_hitters_1.nsmallest(1, '하위타선 합산 지표')
            bottom_hitters_1 = bottom_hitters_1.drop(lowest_score_player.index)
            # 해당 포지션의 선수 추가
            position_candidate = filtered_df[filtered_df['포지션'].str.contains(position)].nlargest(1, '타수')
            bottom_hitters_1 = pd.concat([bottom_hitters_1, position_candidate])

bottom_hitters_1 = bottom_hitters_1.nlargest(4, '하위타선 합산 지표').drop_duplicates()
bottom_hitters_1

,선수명,팀명,타율,경기,타수,홈런,득점,장타율,출루율,득점권타율,...,장타율_표준화,득점권 타율_표준화,홈런_표준화,중심타선 합산 지표,출루율_표준화,도루_표준화,2번타자 합산 지표,1번타자 합산 지표,수비_표준화,하위타선 합산 지표
7,고승민,롯데,0.309,54,204,5,37,0.466,0.378,0.339,...,0.745390,1.048359,0.656532,1.056496,1.034135,0.130605,1.115584,0.405047,0.415636,0.403511
9,나승엽,롯데,0.298,51,171,1,23,0.456,0.381,0.317,...,0.643437,0.798269,-0.919145,0.044688,1.087117,-0.457116,0.010902,0.229643,-2.434383,0.137053
10,박승욱,롯데,0.273,72,187,4,27,0.396,0.344,0.217,...,0.031719,-0.338506,0.262613,0.049947,0.433669,-0.310186,0.211077,0.049423,-0.533613,0.098423
18,유강남,롯데,0.191,52,136,5,11,0.324,0.275,0.234,...,-0.702343,-0.145255,0.656532,-0.003354,-0.784922,-0.457116,-0.380991,-0.425571,2.431225,-0.031600


In [15]:
# bottom_hitters의 선수명을 df2에서 찾기
bottom_hitters_names = bottom_hitters_1['선수명'].values
filtered_df2 = df2[df2['선수명'].isin(bottom_hitters_names)]

# '타순'이 '6-9번' 인 것만 남기기
filtered_df2 = filtered_df2[filtered_df2['타순'].isin(['6번', '7번', '8번', '9번'])]

bottom_hitters = filtered_df2
# bottom_hitters_1에서 '포지션' 컬럼만 선택
bottom_hitters_1_position = bottom_hitters_1[['선수명', '포지션']]

# '선수명'을 기준으로 병합
bottom_hitters = pd.merge(bottom_hitters, bottom_hitters_1_position, on='선수명', how='left')

bottom_hitters

,선수명,팀명,타율,타수,안타,2루타,3루타,홈런,타점,볼넷,...,도루저지,고의4구,희생플라이,희생번트,수비 승리 기여도,득점권타율,추정득점,출루율,장타율,포지션
0,박승욱,롯데,0.333,24,8,2,0,1,5,2,...,4,0,1,2,-0.183,0.217,4.262,0.393,0.542,"유격수, 3루수, 2루수"
1,나승엽,롯데,0.236,55,13,3,1,1,6,10,...,0,0,2,0,-0.601,0.317,9.236,0.353,0.382,1루수
2,유강남,롯데,0.158,19,3,0,0,1,2,1,...,0,0,1,2,0.469,0.234,1.396,0.190,0.316,포수
3,나승엽,롯데,0.333,21,7,3,0,0,0,3,...,0,0,2,0,-0.601,0.317,4.612,0.385,0.476,1루수
4,유강남,롯데,0.240,25,6,1,0,2,5,1,...,0,0,1,2,0.469,0.234,4.464,0.286,0.520,포수
5,박승욱,롯데,0.230,61,14,2,0,2,10,7,...,4,0,1,2,-0.183,0.217,6.992,0.314,0.361,"유격수, 3루수, 2루수"
6,박승욱,롯데,0.286,28,8,2,0,1,4,2,...,4,0,1,2,-0.183,0.217,3.902,0.344,0.464,"유격수, 3루수, 2루수"
7,유강남,롯데,0.235,68,16,2,0,2,13,4,...,0,0,1,2,0.469,0.234,8.678,0.329,0.353,포수
8,박승욱,롯데,0.281,64,18,2,1,0,5,4,...,4,0,1,2,-0.183,0.217,5.998,0.329,0.344,"유격수, 3루수, 2루수"
9,유강남,롯데,0.067,15,1,0,0,0,0,2,...,0,0,1,2,0.469,0.234,-0.008,0.167,0.067,포수


In [16]:
# 각 선수명에 따른 타순 값을 리스트로 모음
bottom_per_player = filtered_df2.groupby('선수명')['타순'].apply(list).reset_index()
bottom_per_player

,선수명,타순
0,고승민,"[6번, 7번, 8번, 9번]"
1,나승엽,"[6번, 7번, 8번, 9번]"
2,박승욱,"[6번, 7번, 8번, 9번]"
3,유강남,"[6번, 7번, 8번, 9번]"


## 만들어진 라인업 결합

### 상위타선

In [17]:
# first_hitter와 second_hitter 병합
df = pd.concat([first_hitter, second_hitter], ignore_index=True)
df

,선수명,팀명,타율,타수,안타,2루타,3루타,홈런,타점,볼넷,...,도루저지,고의4구,희생플라이,희생번트,수비 승리 기여도,득점권타율,추정득점,출루율,장타율,포지션
0,윤동희,롯데,0.264,159,42,10,1,3,17,22,...,2,0,3,1,-0.130,0.348,23.956,0.348,0.396,"중견수, 우익수"
1,황성빈,롯데,0.542,24,13,0,2,3,8,3,...,3,0,0,1,0.015,0.296,14.886,0.593,1.083,"중견수, 좌익수, 우익수"


### 중심타선

In [18]:
# 선수별 가능한 타순을 딕셔너리로 변환
players = center_per_player.set_index('선수명')['타순'].to_dict()

# 가능한 타순 조합 생성
possible_orders = list(itertools.product(*players.values()))

# 타순이 중복되지 않는 조합 필터링
valid_orders = [order for order in possible_orders if len(set(order)) == len(players)]

# 타순과 선수 이름을 매칭하여 조합 생성
combinations = []
for order in valid_orders:
    order_dict = {order[i]: player for i, player in enumerate(players.keys())}
    combinations.append(order_dict)

# 각 조합을 center_hitters에서 찾아오기
results = []
for comb in combinations:
    filtered_center_hitters = center_hitters[center_hitters.apply(lambda row: row['타순'] in comb and comb[row['타순']] == row['선수명'], axis=1)]
    if len(filtered_center_hitters) == len(players):  # 모든 타순이 일치하는 경우
        results.append(filtered_center_hitters)

# 결과를 각각 center_1, center_2 등의 DataFrame으로 저장
center_dfs = []
for i, result in enumerate(results):
    center_df = result.reset_index(drop=True)
    center_dfs.append(center_df)
    globals()[f'center_{i+1}'] = center_df

In [19]:
# center_dfs와 df를 각각 concat
concat_dfs = []
for i, center_df in enumerate(center_dfs):
    concat_df = pd.concat([df, center_df], ignore_index=True)
    concat_dfs.append(concat_df)
    globals()[f'df_{i+1}'] = concat_df

### 하위타선

In [20]:
# 선수별 가능한 타순을 딕셔너리로 변환
players = bottom_per_player.set_index('선수명')['타순'].to_dict()

# 가능한 타순 조합 생성
possible_orders = list(itertools.product(*players.values()))

# 타순이 중복되지 않는 조합 필터링
valid_orders = [order for order in possible_orders if len(set(order)) == len(players)]

# 타순과 선수 이름을 매칭하여 조합 생성
combinations = []
for order in valid_orders:
    order_dict = {order[i]: player for i, player in enumerate(players.keys())}
    combinations.append(order_dict)

# 각 조합을 bottom_hitters에서 찾아오기
results = []
for comb in combinations:
    filtered_bottom_hitters = bottom_hitters[bottom_hitters.apply(lambda row: row['타순'] in comb and comb[row['타순']] == row['선수명'], axis=1)]
    if len(filtered_bottom_hitters) == len(players):  # 모든 타순이 일치하는 경우
        results.append(filtered_bottom_hitters)

# 결과를 각각 bottom_1, bottom_2 등의 DataFrame으로 저장
bottom_dfs = []
for i, result in enumerate(results):
    bottom_df = result.reset_index(drop=True)
    bottom_dfs.append(bottom_df)
    globals()[f'bottom_{i+1}'] = bottom_df

In [21]:
# 모든 bottom_n과 df_n을 각각 concat하여 n*n개의 DataFrame 생성 및 출력
final_dfs = []
count = 1
for i in range(len(concat_dfs)):
    for j in range(len(bottom_dfs)):
        final_df = pd.concat([concat_dfs[i], bottom_dfs[j]], ignore_index=True)
        final_dfs.append(final_df)
        globals()[f'final_df_{count}'] = final_df
        count += 1

# final_df_n의 이름을 df_n으로 변경
for i in range(1, len(final_dfs) + 1):
    globals()[f'df_{i}'] = globals().pop(f'final_df_{i}')

## 베스트 라인업

In [22]:
# 모델 생성
df = pd.read_csv('/content/drive/MyDrive/DATATHON/data/@KBO_5개구단/KBO_top5.csv', encoding='cp949')

df = df.drop(columns=['선수명', '년도'])

# '타율' 열에서 '-' 값을 가진 행을 제거
df = df[df['타율'] != '-']

# 나머지 열에서 '-' 값을 0으로 변경
df.replace('-', 0, inplace=True)

# 특징과 타겟 설정
features = ['출루율', '장타율', '홈런', '수비 승리 기여도', '득점권타율', '도루허용']
target = '추정득점'

X = df[features]
y = df[target]

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# 모델 학습
model = RandomForestRegressor(n_estimators=50, max_depth=20, max_features=0.3, random_state=42)
model.fit(X_train, y_train)

# 모든 df_1부터 df_144까지의 라인업에 대한 추정득점 예측
predicted_scores = {}

for i in range(1, 145):
    lineup_df = globals()[f'df_{i}']
    lineup_features = lineup_df[features]
    predicted_score = model.predict(lineup_features)
    total_predicted_score = predicted_score.mean()  # 각 선수의 예측 점수의 평균
    predicted_scores[f'df_{i}'] = total_predicted_score

# 결과 출력
# for lineup, score in predicted_scores.items():
#    print(f"{lineup}: {score}")

# 점수가 가장 높은 라인업 찾기
best_lineup = max(predicted_scores, key=predicted_scores.get)
best_score = predicted_scores[best_lineup]

# 인덱스를 1번부터 9번까지로 변경
best_lineup_df = globals()[best_lineup].copy()
best_lineup_df.index = range(1, len(best_lineup_df) + 1)

print(f"\n가장 높은 점수를 받은 라인업: {best_lineup} with score {best_score}")
best_lineup_df


가장 높은 점수를 받은 라인업: df_21 with score 46.482


,선수명,팀명,타율,타수,안타,2루타,3루타,홈런,타점,볼넷,...,도루저지,고의4구,희생플라이,희생번트,수비 승리 기여도,득점권타율,추정득점,출루율,장타율,포지션
1,윤동희,롯데,0.264,159,42,10,1,3,17,22,...,2,0,3,1,-0.130,0.348,23.956,0.348,0.396,"중견수, 우익수"
2,황성빈,롯데,0.542,24,13,0,2,3,8,3,...,3,0,0,1,0.015,0.296,14.886,0.593,1.083,"중견수, 좌익수, 우익수"
3,레이예스,롯데,0.335,158,53,9,1,5,29,14,...,0,0,7,0,-0.152,0.398,30.638,0.378,0.500,"우익수, 좌익수, 중견수"
4,전준우,롯데,0.318,154,49,11,1,6,30,14,...,1,0,1,0,-0.269,0.255,27.992,0.380,0.519,좌익수
5,손호영,롯데,0.333,60,20,5,1,3,13,4,...,4,1,1,1,-0.172,0.382,11.860,0.379,0.600,"3루수, 2루수, 유격수"
6,박승욱,롯데,0.333,24,8,2,0,1,5,2,...,4,0,1,2,-0.183,0.217,4.262,0.393,0.542,"유격수, 3루수, 2루수"
7,나승엽,롯데,0.333,21,7,3,0,0,0,3,...,0,0,2,0,-0.601,0.317,4.612,0.385,0.476,1루수
8,유강남,롯데,0.235,68,16,2,0,2,13,4,...,0,0,1,2,0.469,0.234,8.678,0.329,0.353,포수
9,고승민,롯데,0.297,101,30,6,0,2,15,12,...,2,0,4,0,0.026,0.339,18.517,0.367,0.453,"2루수, 좌익수, 우익수, 1루수"


In [23]:
# 각 포지션 개수 세기
from collections import Counter

# 모든 포지션을 담을 리스트
all_positions = []

# 각 선수의 포지션을 분리하여 리스트에 추가
for positions in best_lineup_df['포지션']:
    all_positions.extend(positions.split(', '))

# 각 포지션의 개수 계산
position_counts = Counter(all_positions)

# 결과 출력
print(position_counts)

Counter({'우익수': 4, '좌익수': 4, '중견수': 3, '2루수': 3, '3루수': 2, '유격수': 2, '1루수': 2, '포수': 1})
